In [1]:
import pandas as pd
import pysam

from utils import get_sequences_cagi5

fasta_file = 'data/remote_data/hg19.genome.fa'

In [3]:
df = pd.read_csv('data/cagi5_df.csv')
df.head()

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element
0,X,138612669,T,A,-0.17,0.07,0,release_F9
1,X,138612669,T,C,-0.26,0.24,-1,release_F9
2,X,138612669,T,G,0.34,0.05,0,release_F9
3,X,138612670,A,C,0.00,0.00,0,release_F9
4,X,138612670,A,G,0.22,0.20,1,release_F9


In [3]:
df[df['Pos'].isin([128413071, 128413072, 128413073, 128413074, 128413075])]

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element
2517,8,128413073,C,A,0.09,0.00,0,release_MYCrs6983267
2518,8,128413073,C,T,-0.03,0.00,0,release_MYCrs6983267
2519,8,128413074,C,A,0.04,0.01,0,release_MYCrs6983267
2520,8,128413074,C,G,-0.01,0.00,0,release_MYCrs6983267
2521,8,128413074,C,T,-0.49,0.13,-1,release_MYCrs6983267
2522,8,128413075,T,A,0.00,0.00,0,release_MYCrs6983267
2523,8,128413075,T,C,-0.06,0.01,0,release_MYCrs6983267
2524,8,128413075,T,G,-0.21,0.04,0,release_MYCrs6983267


In [16]:
with pysam.Fastafile(fasta_file) as genome:
  d0 = genome.fetch('chr8', 128413072, 128413073)
  d1 = genome.fetch('chr8', 128413073, 128413074)
  d2 = genome.fetch('chr8', 128413074, 128413075)
  print(d0, d1, d2)

G C T


In [6]:
df[df['Pos'].isin([396142])]

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element
1034,6,396142,T,A,-0.02,0.00,0,release_IRF4
1035,6,396142,T,C,0.02,0.00,0,release_IRF4
1036,6,396142,T,G,0.15,0.07,0,release_IRF4


In [5]:
with pysam.Fastafile(fasta_file) as genome:
  d0 = genome.fetch('chr6', 396141, 396142)
  d1 = genome.fetch('chr6', 396142, 396143)
  print(d0, d1)

G G


In [4]:
get_sequences_cagi5(df)

G does not match row ref T, position 396142 chr 6 in release_IRF4 CRE
0
G does not match row ref T, position 396142 chr 6 in release_IRF4 CRE
0
G does not match row ref T, position 396142 chr 6 in release_IRF4 CRE
0
G does not match row ref C, position 128413073 chr 8 in release_MYCrs6983267 CRE
0
G does not match row ref C, position 128413073 chr 8 in release_MYCrs6983267 CRE
0
T does not match row ref C, position 37775274 chr 6 in release_ZFAND3 CRE
0
T does not match row ref C, position 37775274 chr 6 in release_ZFAND3 CRE
0


two options with things that don't match reference:
 * just predict 0
 * just use what they say as the ref

In [59]:
fasta_file = 'data/remote_data/hg19.genome.fa'
f9_start = 138612624
f9_end = 138612924
pos = 138612670
rel_pos = pos-f9_start-1
with pysam.Fastafile(fasta_file) as genome:
  t = genome.fetch('chrX', 138612668, 138612669)
  a = genome.fetch('chrX', 138612669, 138612670)
  print(t,a)
  dnastr = genome.fetch('chrX', f9_start, f9_end)
print(dnastr[rel_pos])

T A
A


In [52]:
seqstarts = df.groupby(['regulatory_element'])['Pos'].min()
seqends = df.groupby(['regulatory_element'])['Pos'].max()
print(seqstarts)
print(seqends)

regulatory_element
release_F9              138612669
release_GP1BB            19710804
release_HBB               5248268
release_HBG1              5271051
release_HNF4A            42984207
release_IRF4               396142
release_IRF6            209989199
release_LDLR             11199906
release_MSMB             51549083
release_MYCrs6983267    128413073
release_PKLR            155271234
release_SORT1           109817337
release_TERT-GBM          1295104
release_TERT-HEK293T      1295104
release_ZFAND3           37775274
Name: Pos, dtype: int64
regulatory_element
release_F9              138612924
release_GP1BB            19711139
release_HBB               5248439
release_HBG1              5271274
release_HNF4A            42984444
release_IRF4               396589
release_IRF6            209989694
release_LDLR             11200177
release_MSMB             51549578
release_MYCrs6983267    128413664
release_PKLR            155271649
release_SORT1           109817736
release_TERT-GBM    

In [46]:
seqlengths = {}
for k, v in seqstarts.iteritems():
  seqlengths[k] = seqends.loc[k] - v
seqlengths

{'release_F9': 255,
 'release_GP1BB': 335,
 'release_HBB': 171,
 'release_HBG1': 223,
 'release_HNF4A': 237,
 'release_IRF4': 447,
 'release_IRF6': 495,
 'release_LDLR': 271,
 'release_MSMB': 495,
 'release_MYCrs6983267': 591,
 'release_PKLR': 415,
 'release_SORT1': 399,
 'release_TERT-GBM': 258,
 'release_TERT-HEK293T': 258,
 'release_ZFAND3': 579}

## TODO 
* look up evaluation metrics for multiclass classification (is there a binary metric like the one below)

a multiple linear regression model of log2(RNA) ~ log2(DNA) + N + offset (were RNA and DNA are counts observed for all n tags, N is a binary matrix associating n tags to m sequence variants and offset is the models' offset normalizing total DNA to RNA counts) was used to assign sequence effects. From the fit, m coefficients (corresponding to the columns of matrix N) were obtained and assigned as the effects of each sequence variant (SNV). A confidence score was derived by capping the p-value of the multiple linear regression at 10-50 and scaling the log10-transformed value between 0 and 1 (i.e. 1 corresponding to a p-value of ≤10-50, 0.5 of 10-25, 0 to a p-value of 1). We deemed a confidence score greater or equal to 0.1 (p-value of 10-5) indicates that the SNV "has an expression effect" while <0.1 means "has no expression effect".

So we want to predict confidence (regression bounded between 0 and 1), a categorical effect given by:
 -1 if value < 0 and confidence > 0.1,
 0  if confidence < 0.1
 1 if value > 0 and confidence > 0.1,
and probability values for the assigned classes (this could come from predict_proba for example)

prediction is categorical, but data isn't ... Direction - Estimated variant effect from the experimental data. Categorical prediction of repressive. (submitted value -1)/activating (submit a value 1)/no effect (submit a value 0), i.e. allele effect is increasing reporter expression (activating), reducing expression (repressive), or not significantly different from zero (confidence score below 0.1).

Two approaches to consider:
(i) directly training a classifier on the class labels
(ii) training to predict value, confidence (regressors), and using these to compute a prediction for the class

You can probably calculate $\Pi_{y = \hat{y}}(p(\hat{y}))\times\Pi_{y\neq\hat{y}}(1-p(\hat{y}))$, which is an approximation to the full categorical cross entropy i suppose

In [13]:
def get_class(row):
  val = row['Value']
  conf = row['Confidence']
  if conf < 0.1:
    return 0
  elif val < 0:
    return -1
  elif val > 0:
    return 1

In [15]:
classes = f9_df.apply(lambda row: get_class(row), axis=1)

In [16]:
classes

0      0
1     -1
2      0
3      0
4      1
5      0
6      0
7      1
8      0
9      0
10     0
11     0
12     0
13     0
14     1
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25    -1
26     0
27     0
28     0
29     0
      ..
206    0
207    0
208    0
209    1
210    0
211    0
212    0
213    1
214    0
215    0
216    1
217    0
218    0
219    1
220    0
221    0
222    0
223    0
224    0
225    0
226    0
227    0
228    0
229    0
230    0
231    0
232    0
233    0
234    0
235    0
Length: 236, dtype: int64